# Podcast Recommender System 

#### Author Eu Jin Lee: [LinkedIn](https://www.linkedin.com/in/eeujinlee/) | [GitHub](https://github.com/MissingNA)

![](/capstone/images/expressions_chart.jpg)

## Problem Statement 

The advent of podcasts started in late 2004 with the public's greater accesibility to broadband Internet and portable digital audio playback devices such as the iPod. This allowed people to explore the digital media space by developing episodic series of spoken word through digital audio files that could be downloaded onto your personal device for easy listening. Discussion and content within a podcast can range from carefully scripted to totally improvised. Podcasts combine elaborate and artistic sound production with thematic concerns ranging from scientific research to slice-of-life journalism. [1]

However in recent years, podcasting has become a disruptive and popular medium to invest in and use as a commercial platform. 

As the podcast wars intensifies, major players such as Spotify, Apple's iTunes, Pandora and iHeartRadio are rushing to claim dominance within the media space. And one of the ways that has proven to much success in establishing that dominant position is by developing a reliable and accurate recommendation system for their podcast service platforms. 

Our goal is to build a recommendation system based on the podcast descriptors and categories as driving features for our model to create those recommendations. We have managed to sourced a dataset with over ~1 million + rows of podcast data from Listen Notes, a podcast search engine. Making it one of the most comprehensive podcast databases that you can find on the Internet. 

This recommendation system will then be onboarded onto a front facing website interface that users can interact with by hosting our application on Heroku. 

## Executive Summary

## Table of Contents

- [Data Dictionary](#Data-Dictionary)
- [Import Packages](#Import-Packages)
- [Podcast Data](#Podcast-Data)
    - [Data Import and EDA](#Data-Import-and-EDA)
- [Buiding Recommendation Engine](#Building-Recommendation-Engine)
- [Conclusions and Recommendations](#Conclusions-and-Recommendations)
- [References](#References)

## Import Packages

In [1]:
#Standard Packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Podcast Data 

### Data Import and EDA 

In [2]:
podcasts = pd.read_csv('../data/podcasts.csv')

In [3]:
podcasts.shape

(121175, 9)

In [4]:
podcasts.head()

,uuid,title,image,description,language,categories,website,author,itunes_id
0,8d62d3880db2425b890b986e58aca393,"Ecommerce Conversations, by Practical Ecommerce",http://is4.mzstatic.com/image/thumb/Music6/v4/...,Listen in as the Practical Ecommerce editorial...,English,Technology,http://www.practicalecommerce.com,Practical Ecommerce,874457373
1,cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,http://is4.mzstatic.com/image/thumb/Music71/v4...,On the show we’ll be talking to passionate peo...,English,Tech News | Technology,http://developer.telerik.com/,Telerik,1015556393
2,73626ad1edb74dbb8112cd159bda86cf,SoundtrackAlley,http://is5.mzstatic.com/image/thumb/Music71/v4...,A podcast about soundtracks and movies from my...,English,Podcasting | Technology,https://soundtrackalley.podbean.com,Randy Andrews,1158188937
3,0f50631ebad24cedb2fee80950f37a1a,The Tech M&A Podcast,http://is1.mzstatic.com/image/thumb/Music71/v4...,The Tech M&A Podcast pulls from the best of th...,English,Business News | Technology | Tech News | Business,http://www.corumgroup.com,Timothy Goddard,538160025
4,69580e7b419045839ca07af06cf0d653,"The Tech Informist - For fans of Apple, Google...",http://is4.mzstatic.com/image/thumb/Music62/v4...,The tech news show with two guys shooting the ...,English,Gadgets | Tech News | Technology,http://techinformist.com,The Tech Informist,916080498


In [6]:
podcasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121175 entries, 0 to 121174
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uuid         121175 non-null  object
 1   title        121173 non-null  object
 2   image        121175 non-null  object
 3   description  119832 non-null  object
 4   language     121175 non-null  object
 5   categories   121175 non-null  object
 6   website      120005 non-null  object
 7   author       118678 non-null  object
 8   itunes_id    121175 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 8.3+ MB


#### Languages in Podcast Dataset

In [7]:
podcasts.language.value_counts()

English             99316
German               4316
French               3874
Spanish              3637
Portuguese           1827
Swedish              1698
Chinese              1329
Japanese             1097
Italian               818
Russian               602
Dutch                 531
Norwegian             435
Danish                337
Korean                190
Finnish               157
Polish                145
Estonian              128
Czech                 119
Turkish               116
Catalan                93
Slovenian              79
Hungarian              72
Romanian               39
Galician               37
Ukranian               30
Slovak                 22
Swahili                21
Indonesian             19
Afrikaans              13
Icelandic              13
Basque                 13
Greek                  13
Bulgarian               9
Serbian                 8
Albanian                6
Croatian                5
Abkhazian               3
Irish                   3
Belarusian  

#### Filtering for English podcast 

In [8]:
podcasts = podcasts[podcasts.language == 'English']

#### Removing podcast with missing descriptions and duplicate ids

In [9]:
podcasts = podcasts.dropna(subset=['description'])
podcasts = podcasts.drop_duplicates('itunes_id')

In [10]:
podcasts.description.isnull().sum()

0

#### Filtering for podcast descriptions greater than 20 words 

In [11]:
podcasts['description_length'] = [len(x.description.split()) for _, x in podcasts.iterrows()]

In [12]:
podcasts['description_length'].describe()

count    98173.000000
mean        39.168753
std        107.098547
min          0.000000
25%         11.000000
50%         26.000000
75%         51.000000
max      30157.000000
Name: description_length, dtype: float64

About 25\% of our descriptions have less than 11 words or less. Hence, a fair assumption that these won't serve as good inputs when we build the recommender system. Just to be safe, we are only including descriptions that have at least 20 words in them.

In [13]:
podcasts = podcasts[podcasts.description_length >= 20]

### Initialize favorite podcasts

In [14]:
fav_p = ['99% Invisible']
fav_podcast = podcasts[podcasts.title.isin(fav_p)]
fav_podcast

,uuid,title,image,description,language,categories,website,author,itunes_id,description_length
53021,938de7f853b24bf59c7d199b8fba3950,99% Invisible,http://is5.mzstatic.com/image/thumb/Music19/v4...,"\n Design is everywhere in our lives, per...",English,Design | Arts,http://99percentinvisible.org,Roman Mars,394775318,55


## Building Recommender Engine 

### Recommendations from database

In [15]:
podcasts = podcasts[~podcasts.isin(fav_podcast)]
data = pd.concat([podcasts, fav_podcast], sort = True).reset_index(drop = True)

In [16]:
data['description'] = data['description'].astype(str)

In [17]:
data['description'].shape

(59323,)

In [18]:
%%time
# play around with the max_features to see which gives better recommendations 

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), max_features = 1000, stop_words = "english")
tf_idf = tf.fit_transform(data['description'])
tf_idf

CPU times: user 9.54 s, sys: 315 ms, total: 9.86 s
Wall time: 9.89 s


<59323x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 930926 stored elements in Compressed Sparse Row format>

In [19]:
# timed verbose loop function example
# %%time 
# for i in range(100):
#     if i % 10 == 0:
#         print(i)

In [20]:
%%time 
similarity_matrix = cosine_similarity(tf_idf, tf_idf, dense_output=False)

CPU times: user 37.3 s, sys: 6.58 s, total: 43.9 s
Wall time: 45.7 s


In [ ]:
similarity_matrix.shape

### Top 3 Recommended Podcast Picks

In [23]:
x = data[data.title == '99% Invisible'].index[0]
podcast_similarities = similarity_matrix[x].todense()
similar_idx = podcast_similarities.argsort(axis=None)[:,-4:-1]
for i in range(2,-1,-1):
    print(similar_idx.item(i), '-', data.title[similar_idx.item(i)], '-', data.description[similar_idx.item(i)], '\n')
print('Original - ' + data.title[x] + ' -' + data.description[x])

52420 - Benjamen Walker's Theory of Everything - 
      Personally connecting the dots. All of them.  <a href="http://toe.prx.org">Benjamen Walker's Theory of Everything</a> is a proud member of Radiotopia, from PRX. Learn more at <a href="http://radiotopia.fm/">radiotopia.fm</a>.
     

15376 - Radio Diaries - 
      First-person diaries, sound portraits, and hidden chapters of history from Peabody Award-winning producer Joe Richman and the Radio Diaries team. From teenagers to octogenarians, prisoners to prison guards, bra saleswomen to lighthouse keepers. The extraordinary stories of ordinary life.  <a href="http://www.radiodiaries.org">Radio Diaries</a> is a proud member of Radiotopia, from PRX. Learn more at <a href="http://radiotopia.fm/">radiotopia.fm</a>.
     

4299 - Strangers - From Lea Thau, Peabody award-winning producer and creator of The Moth Podcast and The Moth Radio Hour, comes her new storytelling baby, Strangers, part of KCRW's Independent Producer Project. Each epi

### Generating for whole recommendations for all available podcast data

In [38]:
from nltk.tokenize import word_tokenize

In [56]:
podcasts.categories.tolist()

['Tech News | Technology',
 'Business News | Technology | Tech News | Business',
 'Management & Marketing | Business | Technology | Tech News',
 'Tech News | Technology | News & Politics | Gadgets | Software How-To',
 'Podcasting | Management & Marketing | Tech News | Business | Technology',
 'Tech News | Technology | Management & Marketing | Business | Health',
 'Music | Arts | Technology',
 'Comedy | Podcasting | Technology',
 'Tech News | Health | Technology',
 'Tech News | Software How-To | Technology | Education | Training',
 'Management & Marketing | Business | Investing | Podcasting | Technology',
 'Tech News | Technology',
 'Technology | Tech News',
 'Technology | Gadgets',
 'Gadgets | Technology',
 'Technology',
 'Gadgets | Technology',
 'Arts',
 'Hobbies | Arts | Visual Arts | Games & Hobbies',
 'Business | Education | Arts | Performing Arts',
 'Arts | Literature',
 'Arts | Literature',
 'Management & Marketing | Training | Education | Arts | Business | Design',
 'Performing 

In [58]:
text = str(podcasts.categories.tolist())

In [59]:
# split into words 
tokens = word_tokenize(text)
print(tokens[:100])

['[', "'Tech", 'News', '|', 'Technology', "'", ',', "'Business", 'News', '|', 'Technology', '|', 'Tech', 'News', '|', 'Business', "'", ',', "'Management", '&', 'Marketing', '|', 'Business', '|', 'Technology', '|', 'Tech', 'News', "'", ',', "'Tech", 'News', '|', 'Technology', '|', 'News', '&', 'Politics', '|', 'Gadgets', '|', 'Software', 'How-To', "'", ',', "'Podcasting", '|', 'Management', '&', 'Marketing', '|', 'Tech', 'News', '|', 'Business', '|', 'Technology', "'", ',', "'Tech", 'News', '|', 'Technology', '|', 'Management', '&', 'Marketing', '|', 'Business', '|', 'Health', "'", ',', "'Music", '|', 'Arts', '|', 'Technology', "'", ',', "'Comedy", '|', 'Podcasting', '|', 'Technology', "'", ',', "'Tech", 'News', '|', 'Health', '|', 'Technology', "'", ',', "'Tech", 'News', '|', 'Software', 'How-To']


In [60]:
# remove all tokens that are not alphabetic
words = [word for word in tokens if word.isalpha()]
print(words[:100])

['News', 'Technology', 'News', 'Technology', 'Tech', 'News', 'Business', 'Marketing', 'Business', 'Technology', 'Tech', 'News', 'News', 'Technology', 'News', 'Politics', 'Gadgets', 'Software', 'Management', 'Marketing', 'Tech', 'News', 'Business', 'Technology', 'News', 'Technology', 'Management', 'Marketing', 'Business', 'Health', 'Arts', 'Technology', 'Podcasting', 'Technology', 'News', 'Health', 'Technology', 'News', 'Software', 'Technology', 'Education', 'Training', 'Marketing', 'Business', 'Investing', 'Podcasting', 'Technology', 'News', 'Technology', 'Tech', 'News', 'Gadgets', 'Technology', 'Technology', 'Arts', 'Visual', 'Arts', 'Games', 'Hobbies', 'Education', 'Arts', 'Performing', 'Arts', 'Literature', 'Literature', 'Marketing', 'Training', 'Education', 'Arts', 'Business', 'Design', 'Arts', 'Arts', 'Education', 'Tech', 'News', 'Technology', 'Visual', 'Arts', 'Literature', 'Food', 'Arts', 'Music', 'Performing', 'Arts', 'Arts', 'Music', 'Arts', 'Film', 'Society', 'Culture', 'Come

In [71]:
# pull out unique words and save to a list 
def getUniqueWords(allWords) :
    uniqueWords = [] 
    for i in allWords:
        if not i in uniqueWords:
            uniqueWords.append(i)
    return uniqueWords

new_features = getUniqueWords(words)

# convert list into columns and append to podcast dataframe 
grouper = df.groupby('category')
df = pd.concat([pd.Series(v['value'].tolist(), name=k) for k, v in grouper], axis=1)
# run a loop to check if categories column has the word in each of the new columns 1 for yes 0 for no

['News',
 'Technology',
 'Tech',
 'Business',
 'Marketing',
 'Politics',
 'Gadgets',
 'Software',
 'Management',
 'Health',
 'Arts',
 'Podcasting',
 'Education',
 'Training',
 'Investing',
 'Visual',
 'Games',
 'Hobbies',
 'Performing',
 'Literature',
 'Design',
 'Food',
 'Music',
 'Film',
 'Society',
 'Culture',
 'Comedy',
 'Alternative',
 'Places',
 'Travel',
 'History',
 'Video',
 'Kids',
 'Family',
 'TV',
 'Recreation',
 'College',
 'High',
 'School',
 'Other',
 'Shopping',
 'Personal',
 'Journals',
 'Higher',
 'Philosophy',
 'Spirituality',
 'Sports',
 'Christianity',
 'Judaism',
 'Buddhism',
 'Careers',
 'Religion',
 'Hinduism',
 'Government',
 'Organizations',
 'Natural',
 'Sciences',
 'Science',
 'Medicine',
 'Social',
 'Outdoor',
 'Fitness',
 'Nutrition',
 'Language',
 'Courses',
 'Sexuality',
 'Local',
 'Professional',
 'Automotive',
 'Amateur',
 'Educational',
 'Beauty',
 'Fashion',
 'National',
 'Islam',
 'Aviation',
 'Regional',
 'nan']

In [27]:
for podcast in podcasts.title:
    fav_p = [podcast]
    fav_podcast = podcasts[podcasts.title.isin(fav_p)]
    podcasts = podcasts[~podcasts.isin(fav_podcast)]
    data = pd.concat([podcasts, fav_podcast], sort = True).reset_index(drop = True)
    data['description'] = data['description'].astype(str)
    
    # vectorize podcast descriptions 
    %%time
    tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), max_features = 1000, stop_words = "english")
    tf_idf = tf.fit_transform(data['description'])
    
    ## THIS PART SEEMS INEFFICIENT TO DO EVERYTIME YOU GENERATE A PREDICTION ##
    
    # tokenize and dummify categories column
    
    # append to tf_idf dense matrix
    
    #######################################################################
    
    # generates similarity matrix
    %%time 
    similarity_matrix = cosine_similarity(tf_idf, tf_idf, dense_output=False)
    
    # unpacks information on respective podcast and saves info into a dictionary
    x = data[data.title == podcast].index[0]
    podcast_similarities = similarity_matrix[x].todense()
    similar_idx = podcast_similarities.argsort(axis=None)[:,-4:-1]
    for i in range(2,-1,-1):
        rec_db = {'podcast_id':data.uuid[x],
                  'ref_podcast':data.title[x],
                  'podcast_rec_1_idx':similar_idx.item(i), #QUESTION: figuring out how to alias 3 separate recs into dictionary when its a loop
                  'podcast_rec_1_title':data.title[similar_idx.item(i)],
                  'podcast_rec_1_descriptions':data.description[similar_idx.item(i)]
                  'podcast_category': data.category[x] 
                 }
    
    # convert dictionary to dataframe
    
    # save dataframe to data folder 
        
    

Eat Sleep Code Podcast
The Tech M&A Podcast
Social Media, Mobility, Analytics, Cloud: Tech Show! SMACtalk
Radio Leo (Video LO)
Digital India
Digital Health Today
Drum and Bass Electronic Trap Dubstep Music / EDM Radio - DnB  Bass LIVE : Computers (Live) iTunes
Fire in the Hole Podcast
Freedom Scientific FSCast
FLOSS Weekly (Video HD)
This is Marketing Podcast
Paranoria TX
Packet Pushers - Weekly Show
Rocket
Roadshow Reviews (HQ)
Roadshow Originals (HD)
Roadshow Reviews (HD)
Two Guys Talking Wine
Two Ewes Fiber Adventures: A Podcast about Knitting, Spinning, Weaving, Crochet, Dyeing-- All the Ways to Play with Yarn and Wool
Transform My Dance Studio – The Podcast For Dance Studio Owners
Top 5 Comics Podcast
Write Pack Radio
Open Roll with Jurgen Rudolph - the weekly podcast for the Business Artist
Off Stage and On The Air
Opera After Dark
Future Tech: Almost Here, Round-the-Corner Future Technology Podcast
Podcasts – Heck Yeah Comics!
Professional Book Nerds
Prince Street
Power of Fishn

LHC Podden
Sona 9
T'agrada el blues?
Corsten's Countdown Official Podcast
Eldorado
Entrevistas DavidTuba
Nye Skiver på P8 JAZZ
Salihkunduz.net
Talking Tech
Garaget - Volvo Cars Podcast
24syv Dokumentar
Deeper Dish - Authentic Chicago
Fountain Of Thought
ExtravaGaming
Arts and Letters
365 Days of Sport Podcast
Yours Truly Johnny Dollar – The Great Detectives of Old Time Radio
The Agile Revolution
A Time and a Plate with Claire Thomas
The 85 South Show with Karlous Miller, DC Young Fly and Clayton English
The PodCask - a podcast about whiskey
miketechshow's Podcast
The Art of Paid Traffic | Proven Online Advertising Strategies You Can Implement Today
The Starting Zone: A World of Warcraft Podcast!
The Let's Go Eat Show
The Art of Photography
The Sneaker Podcast
The JogTunes Indie Podcast
The BBQ Central Show
The Dough Roller Money Podcast
The Jennifer Fulwiler Show
The Eater Upsell
The Goal Digger Podcast - Marketing, Social Media, Creative Entrepreneurship, Small Business Strategy and B

The Discovery Show
The Messy Table with Jenn Jewell
The Optional Podcast
The Graceful Cowboy Podcast
The Philosophical Angle
El Amor Que Vale on OnePlace.com
The Point with Chris Cillizza
The Offside Rule
Quick Slants - A New England Patriots Podcast
Are You Ready to OVERCOME Podcast
The Horror of Nachos and Hamantaschen
The Corpse Cast
The Instigator Podcast
The Whine Box
Som Winston Vist Sagde
The Prince Podcast
The Know Fear Cast
The Boondoggle - PCEU
The Nostalgia Boys - Press Start to Continue
The FOX News Rundown
The Web with Kurre and Klapow
A Little Bit of Preaching by Tom
Crescent Project Radio
The Seven Minute Sales Minute
The Biz Connection
The Many Moods of Ben Vaughn hosted by Ben Vaughn
The Wide-Eyed Creative
The Winning Walk with Dr. Ed Young - Daily Radio
The Rough Cut
The Love, Happiness and Success Podcast With Dr. Lisa Marie Bobby
The Niskanen Center Political Research Digest Podcast
Ball Yoga Podcast
33% Pulp
Sure Oak: Digital Marketing, SEO, Online Business Strateg

LWAF
Giant Robots Smashing Into Other Giant Robots
Listen Money Matters - Free your inner financial badass. This is not your father's boring personal finance show.
Two Blokes Talking Tech
Good Day, Sir! Show - Salesforce Podcast
Geekazine
Today in Tech History with Tom Merritt
Microsoft IT Showcase  (MP4) - Channel 9
Overtime
Python Bytes
How Do You Engineer?
HomeTech.fm Podcast
Hit the Mic with The Stacey Harris
HackToStart
Official PlayStation Blogcast
MacVoices Video HD
Mac OS Ken
Knightwise.com Audio Feed.
Kodsnack
Kinda Geeky
Kid Friday - apps, websites, gadgets, games, fun!
Jim Stolze
Human Tech
Real Relationships, by Contactually
Real Estate Success Rocks | Top Producing Agents Who Value Excellence, Personal & Professional Growth
Philadelphia Condominiums in Mark's Words | CenterCityCondos.com
The Tech Ranch- Tech podcast online on latest science and technology news
This Week in Tech (Video HD)
The Perspective Podcast | Grow Your Creative Brand
All About Android (Video LO)
Succe

Seduction Debunked
Serena Guided Meditations
Self Care Spotlight: Exploring Self Care & Authentic Living
SheSez with Linda Grasso
Road To Revamp
V.Fit Podcast
Wilshire Church of Christ
Redemption Community Church
Oishi! Holistic! - Kitchen Talk Podcast
Today in the Word Devotional
Press and Reach | A Podcast on Faith |The Christian Walk |God's Purpose for Our Lives
Pleasant Valley Church Sermons
While We're Here
What Have You
Never Wear Boring Socks: Creativity | Lifestyle | Mindset | Growing Up | College | Adulting
Dreamcatchers District Podcast
ER Vet - Stories from the animal ER on Pet Life Radio (PetLifeRadio.com)
Move. Together. A Twin Cities In Motion Podcast
Heal with Kelly Trach
How To Dance - Online Break Dance Classes For Beginners
How She Ate An Elephant
Friday Fuel
Christ Redeemer PCA Weekly Message Podcast
Footnotes
American Real: Everyone Has A Story
Autoimmune Connections
Iglesia Impacto
Inspired Money
UTM Podcast
Kriess Guyte - Synthsonic Sessions
Kinotes
Musician Podca

...

The American Freedom Party Report with James Kelso
The Rock Church General
The Wow! Signal Podcast
The Charles Tendell Show
The Camp
The Snappening - Wedding Photography Podcast
The History of Bad Ideas Podcast
The Podcast of Nonsensical Gamers
The Docket
Boosting Achievement: The ESL Podcast
Flashpoint
For Reel!
podcheese
재즈 듣는 남자
The Firewater Review
The Reach Podcast with John Smithwick
The Restaurant Technology Guys Podcast brought to you by Custom Business Solutions
The Small Moves Podcast
The Superwoman Chronicles
The Grove
The ABV Network Channel
The Flower Pros Podcast
The Program
The I-O Podcast
The Apollo Podcast
The Crochet Circle
Play Big Brand Bold
Listing Agent Lifestyle - Real Estate Marketing
Speaker Success Podcast
Ajahn Kalyano - Talks for Monastics 2015
The Segregationist Papers
Technically Legal
Cover Story
Relatively Insightful
Monday - Saturday
Sunday Morning Hangover Cure
First Time with BTS?
Shoes On Fire Podcast
Far From Ordinary Podcast
Solid Joys Daily Devoti

Heed Army Podcast
Cool Things Entrepreneurs Do
Free Market Underground Podcast
Digital Altitude
Free Beer and Hot Wings Free Clip of the Day Sample
Frazer Smith
Forgive And Forget
Deep Fried Kale
Daniel and Gian Talk
Does This Hold Up
Dead Air Nation
Drink the Sauce
Dudes in a Comic Shop
Derek and Romaine
Doodie Calls with Doug Mand
Can I Pick Your Brain? Entrepreneur Business Podcast
Extra Extra
Catalyst Sale Podcast: Sales Training | Sales Strategy | B2B | Selling | Marketing
Contracting Officer Podcast: Government Contracting, proposal management, proposal writing, governmental contracting, targeting, contract administration, contract management, subcontracting
Contractor Success Map with Randal DeHart | Contractor Bookkeeping And Accounting Services
Chris Hogan's Retire Inspired
Jesse vs Cancer
Joke Strap - Podcast
J360 Radio
Crazy Sh*t In Real Estate with Leigh Brown
CQ For Global Leaders
Democracy That Delivers
Denise Griffitts - Your Partner In Success!
Dr Marketing Tips Podcast

Coaches’ Notes
Kick A Rock's Two Drink Minimum
SUKSESSPILLEN
Digital to Foot Traffic - Online Marketing Guide for Retail Stores
Dr. Gameshow
Faith4Life Church - Dallas TX
ATP Crypto: Asia Tech Podcast Cryptocurrency and Blockchain
RadioQV
Witch Dad: On the Path
Brotherly Love Sports Podcast
The Morphin Grid
Hollywood Happy Place with Gerry Strauss
Pieces of S#!t
Rabata's podcast
Send Network Podcast
Growing Healthy podcast
Test Crushers Chronicle
Royal Watcher: A Podcast for British Royal Family Enthusiasts
DreadHead In Tech
Kayla Got The 411 Podcast
Star Trek Trek
Libertarian Q & A, with Clyde Myers
Daggers Out Podcast
The Next Tuesday Podcast
Jeff Grant's Evolving Podcast
The Mac Observer's Background Mode
The Screech at NU
The Firnecast.
The Voicebot Podcast
The Virtual Couch
Movie Crush
Class Bravo
HER Conversations
Down The Garden Path Podcast
REVIVE TUOLUMNE
REmediaL Studies
Life In The Cracks
Tales From The Road Podcast
Beyond The Beltway
Good Vibes, Good Life
Born To Write: Lea

Netflix Reviews & Coverage
Alternate Ending - Movie Review Podcast
Activate! The Robot Wars Podcast
ABC Podcast: Alex and Brandon Chat
And A Movie: A Community Retrospective
2-Bit Encryption - A Mr Robot Podcast
Churros  y Palomitas
Film Pulse
Comic Confidential
Cinema Swirl
Fifteen Minutes of Doom
Codex Prime
Cinema Red Pill podcast
CinemaJaw
Fresh Perspective
Cracking The Vault
Cordkillers (All Audio)
Father Knows Best Podcast
Creative Writing Career
Fangirl Fridays
Fanboy News Network Podcast – Fanboy News Network
Flash Diaries
First Light
Four Seasons of Film
Filmsquid
For Your Speculation
FilmFrak: The Podcast
DC on CW
Doctor of the Dead
Do I Still Love It?
DAN FOGLER'S 4d Xperience!
Dance Fevre
Damn Dirty Geeks
Diva of DOOL
Decade Old Podcast
Directing Magic Podcast
Down These Mean Streets (Old Time Radio Detectives)
Doctor Who: Mutter's Spiral Podcast
The Doctor Who Podcast from the PodKast with a K
Cabbage Fairy Conversations
Geek Street Podcast
Gardening Australia
Horror and a

Rise and Hustle
Topic Nintendo Podcast
Nimzowitsch
Divas R Frugal: The Podcast
Freedom to Run: Fitness | Nutrition | Running
Delta Chronicles
Strange Nerve
Pass The Remote
Life. Action. RolePlay! A LARP Podcast
Els homes clàssics
Rebel Saint Dharma
Outside the Box
Journey of the Entrepreneur
Big Money in the 805
IndustrialSage | Helping Manufacturing and Industrial Marketers Drive Revenue Through Digital Marketing
Best Investing Podcasts - 3 Per Week
Las Vegas Podcasters
Y'all Are Crazy With Ashley Schubert
Tragic Missile
Uncommon Sense: the This is True Podcast
Friends Fo' Reels
Legendary Creature - Podcast
Blake up with Cody
Change of Plans – 4thebaby
Homo Superior
Fight Horizon
Mighty Whites Podcast
Critical Podcast
Soul Cafe
Lost In Rio
9Cast
Money Wisdom With Joel Johnson
Flens Lair
More In Common
West Side Words - Pilgrim UCC
Just Paranormal Podcast
Love to Hate
MacPsych on the Move
Hip Hop Enthusiast
Chasing the Essential
City Life Church Luton
247Sports College Football Podcast

In [ ]:
# This website function just pulls from a saved database and prints out recs

### Recommendations for unseen data

In [ ]:
# PROCESS 
# search for podcast in database, if it doesn't appear in the search bar,
# try other option 'Recommend based on Unseen Data'
# I.E: enter your podcast title, short description (preferably 20 words, can be just key words) and the one of these categories (drop down menu)

# possible bug, what if the person enters an existing podcast into the alternative option? case sensi or not,
# stops the person from entering an existing podcast. [something like that~~]
new_podcast_title = 'Wait, Hold Up!' 
new_podcast_description = 'Each week, tune in as Yarel and Jessica interview people you know (or who they think you should know) who are on their grind and learn the ''wait, hold up'' moments that changed their lives. When they''re not with a guest they''re diving into the latest in pop culture & politics.'
new_podcast_categories = 

In [73]:
podcasts.head(1)

,uuid,title,image,description,language,categories,website,author,itunes_id,description_length
1,cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,http://is4.mzstatic.com/image/thumb/Music71/v4...,On the show we’ll be talking to passionate peo...,English,Tech News | Technology,http://developer.telerik.com/,Telerik,1.015556e+09,59.0


In [ ]:
# unseen podcast title, description and category entered
new_podcast_row = pd.DataFrame([new_podcast_title,new_podcast_description,new_podcast_categories], columns=list('title','description','categories'))
# appended to the original podcast dataset and generate predictions as per usual. 
podcasts.append(new_podcast_row)

In [ ]:
# CODING IT OUT

# unseen podcast title, description and category entered

# appended to the original podcast dataset and generate predictions as per usual. 

for podcast in podcasts.title:
    fav_p = [podcast]
    fav_podcast = podcasts[podcasts.title.isin(fav_p)]
    podcasts = podcasts[~podcasts.isin(fav_podcast)]
    data = pd.concat([podcasts, fav_podcast], sort = True).reset_index(drop = True)
    data['description'] = data['description'].astype(str)
    
    # vectorize podcast descriptions 
    %%time
    tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), max_features = 1000, stop_words = "english")
    tf_idf = tf.fit_transform(data['description'])
    
    
    ## THIS PART SEEMS INEFFICIENT TO DO EVERYTIME YOU GENERATE A PREDICTION ##
    
    # tokenize and dummify categories column
    
    # append to tf_idf dense matrix
    
    #######################################################################
    
    # generates similarity matrix
    %%time 
    similarity_matrix = cosine_similarity(tf_idf, tf_idf, dense_output=False)
    
    # unpacks information on respective podcast 
    x = data[data.title == podcast].index[0]
    podcast_similarities = similarity_matrix[x].todense()
    similar_idx = podcast_similarities.argsort(axis=None)[:,-4:-1]
    for i in range(2,-1,-1):
        print(similar_idx.item(i), '-', data.title[similar_idx.item(i)], '-', data.description[similar_idx.item(i)], '\n')
    print('Original - ' + data.title[x] + ' -' + data.description[x])

In [ ]:
# This website function generates a recommendation based on what user has entered and prints out recs

## Conclusions and Recommendations 

## References 


# Expansion 

## Episodes Data 

### Data Import and EDA

In [ ]:
episodes = pd.read_csv('../data/episodes.csv')

In [ ]:
episodes.shape

In [ ]:
episodes.head()

In [ ]:
episodes.info()